In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import folium
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_bars = pd.read_csv('../../data/osm-fr-bars.csv', sep=';')

display(data_bars)

In [ ]:
data_bars = data_bars[["Geo Point", "Code postal", "Nom"]]

display(data_bars)
print("Rate of undefined values:", data_bars["Code postal"].isna().sum() / len(data_bars["Code postal"]))

Since the rate of undefined value is way higher than 0.5, and that GPS coordinates and postal code are a little redundant, we can reasonably delete Code postal column:

In [ ]:
data_bars.drop("Code postal", axis=1, inplace=True)
data_bars

In [ ]:
latitudes = []
longitudes = []

for row in data_bars['Geo Point']:
    try:
        latitudes.append(row.split(',')[0])
        longitudes.append(row.split(',')[1])
    except:
        latitudes.append(np.NaN)
        longitudes.append(np.NaN)

data_bars['Latitude'] = latitudes
data_bars['Longitude'] = longitudes
data_bars.drop('Geo Point', 1)

Number of missing values

In [ ]:
data_bars.isnull().sum().sum()
data_bars

Let's just see that on a map

In [ ]:
postalCodes = pd.read_csv('../../data/postal-code.csv', usecols=['01400', '46.147624', '4.923727'],
                          dtype={'01400': str, '46.147624': np.float64, '4.923727': np.float64})
postalCodes

In [ ]:
postalCodes.columns = ['Postal code', 'Latitude', 'Longitude']
postalCodes

Le meilleur compromis d'arrondi des coordonnées GPS est de 3 car pour un arrondi de 4 nous obtenons 16749 valeurs non trouvées et en dessous de 3 la précision est trop faible : on peut obtenir un code postal erroné. Nous aurions pu faire un appel à une API pour avoir des valeurs exacte facilement, mais nous trouvions que c'était moins dans l'esprit du projet.
9859; 1524; 617;

In [ ]:
postalCodes['Latitude'] = postalCodes['Latitude'].apply(lambda lat: round(lat, 3))
postalCodes['Longitude'] = postalCodes['Longitude'].apply(lambda long: round(long, 3))

data_bars['Latitude'] = data_bars['Latitude'].astype(float)
data_bars['Longitude'] = data_bars['Longitude'].astype(float)
data_bars['Latitude'] = data_bars['Latitude'].apply(lambda lat: round(lat, 3))
data_bars['Longitude'] = data_bars['Longitude'].apply(lambda long: round(long, 3))

data_bars = pd.merge(data_bars, postalCodes, how='left', on=['Latitude', 'Longitude']).drop_duplicates()
data_bars["Postal code"] = data_bars["Postal code"].str.strip()
data_bars

In [ ]:
data_bars.dropna(subset=['Postal code'], inplace=True)
data_bars['Département'] = data_bars['Postal code'].apply(lambda code: str(code)[0:2])
data_bars.drop_duplicates(subset=['Geo Point'], inplace=True)
data_bars

In [ ]:
barNumber = data_bars.groupby(['Département'])['Département'].count().reset_index(name='Nombre de bars')
barNumber

In [ ]:
import plotly.express as px
import json

departements = json.load(open('../../data/departements-version-simplifiee.geojson'))
fig = px.choropleth_mapbox(barNumber, geojson=departements,
                           locations='Département', featureidkey='properties.code',  # join keys
                           color='Nombre de bars', color_continuous_scale=px.colors.sequential.turbid,
                           mapbox_style="carto-positron",
                           range_color=(0, 1000),
                           zoom=4.5, center={"lat": 47, "lon": 2},
                           opacity=0.75,
                           labels={'prix': 'Nombre de bars'}
                           )
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig

In [ ]:
departements = json.load(open('../../data/departements-version-simplifiee.geojson'))

fig = px.choropleth_mapbox(barNumber.sort_values(by='Nombre de bars'), geojson=departements,
                           locations='Département', featureidkey='properties.code',  # join keys
                           color='Nombre de bars', color_continuous_scale=px.colors.sequential.turbid,
                           mapbox_style="carto-positron",
                           range_color=(0, 1000),
                           zoom=4.5, center={"lat": 47, "lon": 2},
                           opacity=0.75,
                           animation_frame='Nombre de bars',
                           labels={'prix': 'Nombre de bars'}
                           )
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig

Clean data_revenus

In [ ]:
#DATA_REVENUS INITIALISATION
data_revenus = pd.read_excel('../../data/revenus_communes_2019.xlsx')  #header=3
data_revenus.drop(data_revenus.columns[[0, 3, 7, 8, 9, 10, 11, 12, 13]], axis=1, inplace=True)
data_revenus.drop(data_revenus.tail(2).index, inplace=True)  # drop last 2 rows
display(data_revenus)

In [ ]:
# On peut clean ca et mettre direct dans le read du fichier (voir pb pour header=3)
data_revenus.rename(columns=lambda n: data_revenus[n][2], inplace=True)
data_revenus.drop(index=[0, 1, 2], inplace=True)
data_revenus

In [ ]:
data_revenus = data_revenus[data_revenus["Revenu fiscal de référence par tranche (en euros)"].str.strip() == 'TOTAL']
data_revenus

In [ ]:
data_revenus = data_revenus[~data_revenus['Dép.'].str.startswith('B')]
data_revenus

In [ ]:
figure = px.density_mapbox(data_bars, lat='Latitude', lon='Longitude', z='Département', radius=4.3,
                           color_continuous_scale=px.colors.sequential.Plasma,
                           zoom=4.5, center={"lat": 47, "lon": 2},
                           mapbox_style="stamen-terrain", )

figure.update_layout(mapbox_style="carto-darkmatter")
figure.update_layout(margin=dict(b=0, t=0, l=0, r=0))
figure.show()

In [ ]:
figure = px.density_mapbox(data_bars, lat='Latitude', lon='Longitude', z='Département', radius=4.3,
                           animation_frame='Département',
                           zoom=4.5, center={"lat": 47, "lon": 2},
                           mapbox_style="stamen-terrain", )

figure.update_layout(mapbox_style="carto-darkmatter")
figure.update_layout(margin=dict(b=0, t=0, l=0, r=0))
figure.show()

In [ ]:
data_revenus.insert(1, "Code commune",
                    data_revenus['Dép.'].str.strip().apply(lambda dep: dep[:2]) + data_revenus['Commune'])
data_revenus.drop(["Dép.", "Commune"], axis=1, inplace=True)
data_revenus['Code commune'] = data_revenus['Code commune'].str.strip()
data_revenus

In [ ]:
code_commune = pd.read_csv('../../data/code-commune.csv', sep=';')
code_commune.drop(['Nom_commune', 'Ligne_5', 'Libellé_d_acheminement', 'coordonnees_gps'], axis=1, inplace=True)
code_commune.columns = ['Code commune', 'Postal code']
code_commune['Postal code'] = code_commune['Postal code'].astype(str)
code_commune['Postal code'] = code_commune['Postal code'].apply(
    lambda postalCode: '0' + postalCode if (len(postalCode) < 5) else postalCode).str.strip()
code_commune

In [ ]:
data_revenus = pd.merge(data_revenus, code_commune, on=['Code commune'], how='left').drop(
    ['Revenu fiscal de référence par tranche (en euros)'], axis=1)
data_revenus

In [ ]:
data_revenus = data_revenus.drop_duplicates()
data_revenus = data_revenus.reset_index(drop=True)
data_revenus

In [ ]:
data_bars_revenus = pd.merge(data_bars, data_revenus, on=['Postal code'], how='left')
data_bars_revenus

In [ ]:
data_bars_revenus.drop(
    data_bars_revenus[data_bars_revenus['Revenu fiscal de référence des foyers fiscaux'] == 'n.c.'].index, inplace=True)
data_bars_revenus.drop_duplicates(inplace=True)
data_bars_revenus['Revenu fiscal de référence des foyers fiscaux'] = data_bars_revenus[
    'Revenu fiscal de référence des foyers fiscaux'].astype(np.float64)
data_bars_revenus['Revenu fiscal de référence par foyer fiscal'] = data_bars_revenus['Revenu fiscal de référence des foyers fiscaux']/data_bars_revenus['Nombre de foyers fiscaux']
data_bars_revenus

In [ ]:
nbBar_revenu_byCp = pd.merge(
    data_bars_revenus.groupby('Postal code')['Revenu fiscal de référence par foyer fiscal'].mean().reset_index(),
    data_bars_revenus.groupby('Postal code')['Nom'].nunique().reset_index(), on=['Postal code'], how='inner')
nbBar_revenu_byCp.columns = ['Postal code', 'Revenu fiscal de référence par foyer fiscal', 'Nombre de bars']
nbBar_revenu_byCp

In [ ]:
data_bars_revenus_unique = data_bars_revenus.iloc[:, :6].drop_duplicates().merge(
    nbBar_revenu_byCp[['Postal code', 'Revenu fiscal de référence par foyer fiscal']], on=['Postal code'], how='inner')

figure = px.density_mapbox(data_bars_revenus_unique, lat='Latitude', lon='Longitude',
                           z='Revenu fiscal de référence par foyer fiscal', radius=5,
                           color_continuous_scale=px.colors.carto.Temps,
                           zoom=4.5, center={"lat": 47, "lon": 2},
                           mapbox_style="stamen-terrain", )

figure.update_layout(mapbox_style="carto-darkmatter")
figure.update_layout(margin=dict(b=0, t=0, l=0, r=0))
figure.show()

In [ ]:
departements = json.load(open('../../data/contours-codes-postaux.geojson'))
tmp = nbBar_revenu_byCp.drop('Revenu fiscal de référence par foyer fiscal', axis=1)
tmp['Revenu par foyer fiscal'] = nbBar_revenu_byCp['Revenu fiscal de référence par foyer fiscal'].round(decimals=1)

fig = px.choropleth_mapbox(tmp, geojson=departements,
                           locations='Postal code', featureidkey='properties.codePostal',  # join keys
                           color='Nombre de bars', color_continuous_scale=px.colors.carto.Redor,
                           mapbox_style="carto-positron",
                           hover_data=['Revenu par foyer fiscal'],
                           range_color=(0, 150),
                           zoom=10, center={"lat": 48.857381, "lon": 2.34453},
                           opacity=0.75,
                           labels={'prix': 'Nombre de bars'}
                           )
fig.update_layout(mapbox_style="carto-darkmatter", margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig

In [ ]:
ig = px.choropleth_mapbox(tmp, geojson=departements,
                          locations='Postal code', featureidkey='properties.codePostal',  # join keys
                          color='Nombre de bars', color_continuous_scale=px.colors.carto.Redor,
                          mapbox_style="carto-positron",
                          hover_data=['Revenu par foyer fiscal'],
                          range_color=(0, 150),
                          zoom=10, center={"lat": 48.857381, "lon": 2.34453},
                          opacity=0.75,
                          labels={'prix': 'Nombre de bars'}
                          )
fig.update_layout(mapbox_style="carto-darkmatter", margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig

In [ ]:
g = sns.relplot(data=nbBar_revenu_byCp, x='Nombre de bars', y='Revenu fiscal de référence par foyer fiscal')
g.set(yscale='log')  # same, without the log scale we cannot see anything

In [ ]:
mean_by_bar_number = nbBar_revenu_byCp.groupby('Nombre de bars').mean().reset_index()
mean_by_bar_number

In [ ]:
plt.plot(mean_by_bar_number['Nombre de bars'], mean_by_bar_number['Revenu fiscal de référence par foyer fiscal'])
plt.xlabel('Nombre de bars')
plt.ylabel('Revenu fiscal de référence moyen')

In [ ]:
fig = px.histogram(data_frame=mean_by_bar_number, x='Revenu fiscal de référence par foyer fiscal', y='Nombre de bars')
fig.show()

In [ ]:
g = sns.relplot(data=mean_by_bar_number, y='Revenu fiscal de référence par foyer fiscal',
                x=mean_by_bar_number['Nombre de bars'])
g.set(yscale='log')  # same, without the log scale we cannot see anything

In [ ]:
nbBar_revenu_byCp = data_bars_revenus.groupby('Nom')['Revenu fiscal de référence par foyer fiscal'].mean().reset_index()

In [ ]:
g = sns.relplot(data=nbBar_revenu_byCp, x='Revenu fiscal de référence par foyer fiscal', y=nbBar_revenu_byCp.index)
g.set(yscale='log')  # same, without the log scale we cannot see anything

In [ ]:
plt.scatter(nbBar_revenu_byCp.index, nbBar_revenu_byCp['Revenu fiscal de référence par foyer fiscal'], label='nomalies')
plt.title('Connexions en fonction de l\'instant dans le temps et la durée de la connexion')
plt.ylabel('Revenu fiscal de référence moyen')
plt.xlabel('Bar n°')
plt.show()

In [ ]:
ax = sns.lineplot(x=np.arange(len(nbBar_revenu_byCp)),
                  y=np.sort(nbBar_revenu_byCp['Revenu fiscal de référence par foyer fiscal']))
ax.set_title('Revenu fiscal de référence des %d bars' % len(nbBar_revenu_byCp))
ax.set(yscale='log', xlabel='Nombre de bars', ylabel='Revenu fiscal de référence par foyer fiscal')

In [6]:
data_bars.isnull().sum().sum()
data_bars

,Geo Point,Nom,Latitude,Longitude
0,"44.0549704,0.2603921",Au boin coin,44.0549704,0.2603921
1,"47.9094426,1.9076031",Le Diabolo Menthe,47.9094426,1.9076031
2,"48.8020856,2.427514",Café de la Gare,48.8020856,2.427514
3,"41.6976671,9.0644945",NaN,41.6976671,9.0644945
4,"43.788717,4.8299152",Brasserie du Commerce,43.788717,4.8299152
...,...,...,...,...
17928,"44.2761336,5.2758837",Les Glycines,44.2761336,5.2758837
17929,"46.7227908,2.5037666",La Rotonde,46.7227908,2.5037666
17930,"43.8339541,4.3534613",La Movida,43.8339541,4.3534613
17931,"46.7776345,4.8488139",Nicéphore Café,46.7776345,4.8488139


Let's just see that on a map

In [7]:
postalCodes = pd.read_csv('../../data/postal-code.csv', usecols=['01400', '46.147624', '4.923727'],
                          dtype={'01400': str, '46.147624': np.float64, '4.923727': np.float64})
postalCodes

,01400,46.147624,4.923727
0,01400,46.147812,4.924310
1,01400,46.146756,4.924307
2,01400,46.146907,4.924547
3,01400,46.147154,4.924191
4,01400,46.147307,4.924624
...,...,...,...
22845076,97680,-12.790557,45.137112
22845077,97680,-12.789817,45.132364
22845078,97680,-12.788548,45.132566
22845079,97680,-12.789423,45.103516


In [8]:
postalCodes.columns = ['Postal code', 'Latitude', 'Longitude']
postalCodes

,Postal code,Latitude,Longitude
0,01400,46.147812,4.924310
1,01400,46.146756,4.924307
2,01400,46.146907,4.924547
3,01400,46.147154,4.924191
4,01400,46.147307,4.924624
...,...,...,...
22845076,97680,-12.790557,45.137112
22845077,97680,-12.789817,45.132364
22845078,97680,-12.788548,45.132566
22845079,97680,-12.789423,45.103516


Le meilleur compromis d'arrondi des coordonnées GPS est de 3 car pour un arrondi de 4 nous obtenons 16749 valeurs non trouvées et en dessous de 3 la précision est trop faible : on peut obtenir un code postal erroné. Nous aurions pu faire un appel à une API pour avoir des valeurs exacte facilement, mais nous trouvions que c'était moins dans l'esprit du projet.
9859; 1524; 617;

In [9]:
postalCodes['Latitude'] = postalCodes['Latitude'].apply(lambda lat: round(lat, 3))
postalCodes['Longitude'] = postalCodes['Longitude'].apply(lambda long: round(long, 3))

data_bars['Latitude'] = data_bars['Latitude'].astype(float)
data_bars['Longitude'] = data_bars['Longitude'].astype(float)
data_bars['Latitude'] = data_bars['Latitude'].apply(lambda lat: round(lat, 3))
data_bars['Longitude'] = data_bars['Longitude'].apply(lambda long: round(long, 3))

data_bars = pd.merge(data_bars, postalCodes, how='left', on=['Latitude', 'Longitude']).drop_duplicates()
data_bars["Postal code"] = data_bars["Postal code"].str.strip()
data_bars

KeyboardInterrupt: 

In [ ]:
data_bars.dropna(subset=['Postal code'], inplace=True)
data_bars['Département'] = data_bars['Postal code'].apply(lambda code: str(code)[0:2])
data_bars.drop_duplicates(subset=['Geo Point'], inplace=True)
data_bars

In [ ]:
barNumber = data_bars.groupby(['Département'])['Département'].count().reset_index(name='Nombre de bars')
barNumber

In [ ]:
import plotly.express as px
import json

departements = json.load(open('../../data/departements-version-simplifiee.geojson'))
fig = px.choropleth_mapbox(barNumber, geojson=departements,
                           locations='Département', featureidkey='properties.code',  # join keys
                           color='Nombre de bars', color_continuous_scale=px.colors.sequential.turbid,
                           mapbox_style="carto-positron",
                           range_color=(0, 1000),
                           zoom=4.5, center={"lat": 47, "lon": 2},
                           opacity=0.75,
                           labels={'prix': 'Nombre de bars'}
                           )
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig

In [ ]:
departements = json.load(open('../../data/departements-version-simplifiee.geojson'))

fig = px.choropleth_mapbox(barNumber.sort_values(by='Nombre de bars'), geojson=departements,
                           locations='Département', featureidkey='properties.code',  # join keys
                           color='Nombre de bars', color_continuous_scale=px.colors.sequential.turbid,
                           mapbox_style="carto-positron",
                           range_color=(0, 1000),
                           zoom=4.5, center={"lat": 47, "lon": 2},
                           opacity=0.75,
                           animation_frame='Nombre de bars',
                           labels={'prix': 'Nombre de bars'}
                           )
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig

Clean data_revenus

In [ ]:
#DATA_REVENUS INITIALISATION
data_revenus = pd.read_excel('../../data/revenus_communes_2019.xlsx')  #header=3
data_revenus.drop(data_revenus.columns[[0, 3, 7, 8, 9, 10, 11, 12, 13]], axis=1, inplace=True)
data_revenus.drop(data_revenus.tail(2).index, inplace=True)  # drop last 2 rows
display(data_revenus)

In [ ]:
# On peut clean ca et mettre direct dans le read du fichier (voir pb pour header=3)
data_revenus.rename(columns=lambda n: data_revenus[n][2], inplace=True)
data_revenus.drop(index=[0, 1, 2], inplace=True)
data_revenus

In [ ]:
data_revenus = data_revenus[data_revenus["Revenu fiscal de référence par tranche (en euros)"].str.strip() == 'TOTAL']
data_revenus

In [ ]:
data_revenus = data_revenus[~data_revenus['Dép.'].str.startswith('B')]
data_revenus

In [ ]:
figure = px.density_mapbox(data_bars, lat='Latitude', lon='Longitude', z='Département', radius=4.3,
                           color_continuous_scale=px.colors.sequential.Plasma,
                           zoom=4.5, center={"lat": 47, "lon": 2},
                           mapbox_style="stamen-terrain", )

figure.update_layout(mapbox_style="carto-darkmatter")
figure.update_layout(margin=dict(b=0, t=0, l=0, r=0))
figure.show()

In [ ]:
figure = px.density_mapbox(data_bars, lat='Latitude', lon='Longitude', z='Département', radius=4.3,
                           animation_frame='Département',
                           zoom=4.5, center={"lat": 47, "lon": 2},
                           mapbox_style="stamen-terrain", )

figure.update_layout(mapbox_style="carto-darkmatter")
figure.update_layout(margin=dict(b=0, t=0, l=0, r=0))
figure.show()

In [ ]:
data_revenus.insert(1, "Code commune",
                    data_revenus['Dép.'].str.strip().apply(lambda dep: dep[:2]) + data_revenus['Commune'])
data_revenus.drop(["Dép.", "Commune"], axis=1, inplace=True)
data_revenus['Code commune'] = data_revenus['Code commune'].str.strip()
data_revenus

In [ ]:
code_commune = pd.read_csv('../../data/code-commune.csv', sep=';')
code_commune.drop(['Nom_commune', 'Ligne_5', 'Libellé_d_acheminement', 'coordonnees_gps'], axis=1, inplace=True)
code_commune.columns = ['Code commune', 'Postal code']
code_commune['Postal code'] = code_commune['Postal code'].astype(str)
code_commune['Postal code'] = code_commune['Postal code'].apply(
    lambda postalCode: '0' + postalCode if (len(postalCode) < 5) else postalCode).str.strip()
code_commune

In [ ]:
data_revenus = pd.merge(data_revenus, code_commune, on=['Code commune'], how='left').drop(
    ['Revenu fiscal de référence par tranche (en euros)'], axis=1)
data_revenus

In [ ]:
data_revenus = data_revenus.drop_duplicates()
data_revenus = data_revenus.reset_index(drop=True)
data_revenus

In [ ]:
data_bars_revenus = pd.merge(data_bars, data_revenus, on=['Postal code'], how='left')
data_bars_revenus

In [ ]:
data_bars_revenus.drop(
    data_bars_revenus[data_bars_revenus['Revenu fiscal de référence des foyers fiscaux'] == 'n.c.'].index, inplace=True)
data_bars_revenus.drop_duplicates(inplace=True)
data_bars_revenus['Revenu fiscal de référence des foyers fiscaux'] = data_bars_revenus[
    'Revenu fiscal de référence des foyers fiscaux'].astype(np.float64)
data_bars_revenus['Revenu fiscal de référence par foyer fiscal'] = data_bars_revenus['Revenu fiscal de référence des foyers fiscaux']/data_bars_revenus['Nombre de foyers fiscaux']
data_bars_revenus

In [ ]:
nbBar_revenu_byCp = pd.merge(
    data_bars_revenus.groupby('Postal code')['Revenu fiscal de référence par foyer fiscal'].mean().reset_index(),
    data_bars_revenus.groupby('Postal code')['Nom'].nunique().reset_index(), on=['Postal code'], how='inner')
nbBar_revenu_byCp.columns = ['Postal code', 'Revenu fiscal de référence par foyer fiscal', 'Nombre de bars']
nbBar_revenu_byCp

In [ ]:
data_bars_revenus_unique = data_bars_revenus.iloc[:, :6].drop_duplicates().merge(
    nbBar_revenu_byCp[['Postal code', 'Revenu fiscal de référence par foyer fiscal']], on=['Postal code'], how='inner')

figure = px.density_mapbox(data_bars_revenus_unique, lat='Latitude', lon='Longitude',
                           z='Revenu fiscal de référence par foyer fiscal', radius=5,
                           color_continuous_scale=px.colors.carto.Temps,
                           zoom=4.5, center={"lat": 47, "lon": 2},
                           mapbox_style="stamen-terrain", )

figure.update_layout(mapbox_style="carto-darkmatter")
figure.update_layout(margin=dict(b=0, t=0, l=0, r=0))
figure.show()

In [ ]:
departements = json.load(open('../../data/contours-codes-postaux.geojson'))
tmp = nbBar_revenu_byCp.drop('Revenu fiscal de référence par foyer fiscal', axis=1)
tmp['Revenu par foyer fiscal'] = nbBar_revenu_byCp['Revenu fiscal de référence par foyer fiscal'].round(decimals=1)

fig = px.choropleth_mapbox(tmp, geojson=departements,
                           locations='Postal code', featureidkey='properties.codePostal',  # join keys
                           color='Nombre de bars', color_continuous_scale=px.colors.carto.Redor,
                           mapbox_style="carto-positron",
                           hover_data=['Revenu par foyer fiscal'],
                           range_color=(0, 150),
                           zoom=10, center={"lat": 48.857381, "lon": 2.34453},
                           opacity=0.75,
                           labels={'prix': 'Nombre de bars'}
                           )
fig.update_layout(mapbox_style="carto-darkmatter", margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig

In [ ]:
ig = px.choropleth_mapbox(tmp, geojson=departements,
                          locations='Postal code', featureidkey='properties.codePostal',  # join keys
                          color='Nombre de bars', color_continuous_scale=px.colors.carto.Redor,
                          mapbox_style="carto-positron",
                          hover_data=['Revenu par foyer fiscal'],
                          range_color=(0, 150),
                          zoom=10, center={"lat": 48.857381, "lon": 2.34453},
                          opacity=0.75,
                          labels={'prix': 'Nombre de bars'}
                          )
fig.update_layout(mapbox_style="carto-darkmatter", margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig

In [ ]:
g = sns.relplot(data=nbBar_revenu_byCp, x='Nombre de bars', y='Revenu fiscal de référence par foyer fiscal')
g.set(yscale='log')  # same, without the log scale we cannot see anything

In [ ]:
mean_by_bar_number = nbBar_revenu_byCp.groupby('Nombre de bars').mean().reset_index()
mean_by_bar_number

In [ ]:
plt.plot(mean_by_bar_number['Nombre de bars'], mean_by_bar_number['Revenu fiscal de référence par foyer fiscal'])
plt.xlabel('Nombre de bars')
plt.ylabel('Revenu fiscal de référence moyen')

In [ ]:
fig = px.histogram(data_frame=mean_by_bar_number, x='Revenu fiscal de référence par foyer fiscal', y='Nombre de bars')
fig.show()

In [ ]:
g = sns.relplot(data=mean_by_bar_number, y='Revenu fiscal de référence par foyer fiscal',
                x=mean_by_bar_number['Nombre de bars'])
g.set(yscale='log')  # same, without the log scale we cannot see anything

In [ ]:
nbBar_revenu_byCp = data_bars_revenus.groupby('Nom')['Revenu fiscal de référence par foyer fiscal'].mean().reset_index()

In [ ]:
g = sns.relplot(data=nbBar_revenu_byCp, x='Revenu fiscal de référence par foyer fiscal', y=nbBar_revenu_byCp.index)
g.set(yscale='log')  # same, without the log scale we cannot see anything

In [ ]:
plt.scatter(nbBar_revenu_byCp.index, nbBar_revenu_byCp['Revenu fiscal de référence par foyer fiscal'], label='nomalies')
plt.title('Connexions en fonction de l\'instant dans le temps et la durée de la connexion')
plt.ylabel('Revenu fiscal de référence moyen')
plt.xlabel('Bar n°')
plt.show()

In [ ]:
ax = sns.lineplot(x=np.arange(len(nbBar_revenu_byCp)),
                  y=np.sort(nbBar_revenu_byCp['Revenu fiscal de référence par foyer fiscal']))
ax.set_title('Revenu fiscal de référence des %d bars' % len(nbBar_revenu_byCp))
ax.set(yscale='log', xlabel='Nombre de bars', ylabel='Revenu fiscal de référence par foyer fiscal')